In [1]:
import cv2
import torch
import torch.nn as nn
from network import Conv2d
from models import CSRNet
from models import MCNN
from crowd_count import CrowdCounter
from data_loader import ImageDataLoader
import torch.optim as optim
import os
import h5py
import matplotlib.pyplot as plt
import numpy as np

In [2]:
csrnet = CrowdCounter('csrnet')
mcnn = CrowdCounter('mcnn')

In [3]:
root = 'C:/Users/bennt/Documents/GitHub/cnn-model/data/SHT/part_A_final/'

train_path = root + 'train_data/consolidated'
train_gt_path = root + 'train_data/ground_truth'
val_path = root + 'test_data/consolidated'
val_gt_path = root + 'test_data/ground_truth'

data_loader = ImageDataLoader(train_path, shuffle=False, pre_load=False)
data_loader_val = ImageDataLoader(val_path, shuffle=False, pre_load=False)

In [4]:
for i in data_loader:
    print(i)

{'data': array([[[[1.        , 0.99607843, 1.        , ..., 0.98431373,
          0.99607843, 0.99607843],
         [0.98823529, 1.        , 0.99607843, ..., 0.8627451 ,
          0.85098039, 0.85490196],
         [0.8745098 , 0.92941176, 0.93333333, ..., 0.92941176,
          0.93333333, 0.94117647],
         ...,
         [0.57254902, 0.56470588, 0.57647059, ..., 0.46666667,
          0.47058824, 0.47843137],
         [0.4627451 , 0.46666667, 0.48235294, ..., 0.5372549 ,
          0.54117647, 0.5254902 ],
         [0.52941176, 0.5372549 , 0.52156863, ..., 0.41568627,
          0.4       , 0.40392157]],

        [[0.54117647, 0.51764706, 0.5254902 , ..., 0.46666667,
          0.47058824, 0.48235294],
         [0.46666667, 0.47058824, 0.48235294, ..., 0.51372549,
          0.52156863, 0.51764706],
         [0.50980392, 0.50980392, 0.50588235, ..., 0.43529412,
          0.41960784, 0.42352941],
         ...,
         [0.81960784, 0.76470588, 0.86666667, ..., 0.90588235,
          0.9137

In [5]:
data_loader

In [7]:
#training configuration
start_step = 0
end_step = 20
lr = 0.00001
momentum = 0.9
disp_interval = 2
log_interval = 250

for epoch in range(start_step, end_step+1):    
    step = -1
    train_loss = 0
    for blob in data_loader:                
        step += 1        
        im_data = blob['data']
        gt_data = blob['gt_density']
        density_map = csrnet(im_data, gt_data)
        
        print(gt_data.sum())
        print(density_map.data.sum())
        
        loss = csrnet.loss
        train_loss += loss.data
        #step_cnt += 1
        #optimizer.zero_grad()
        loss.backward()
        #optimizer.step()
        break
        
        if step % disp_interval == 0:            
            duration = t.toc(average=False)
            fps = step_cnt / duration
            gt_count = np.sum(gt_data)    
            density_map = density_map.data.cpu().numpy()
            et_count = np.sum(density_map)
            utils.save_results(im_data,gt_data,density_map, output_dir)
            log_text = 'epoch: %4d, step %4d, Time: %.4fs, gt_cnt: %4.1f, et_cnt: %4.1f' % (epoch,
                step, 1./fps, gt_count,et_count)
            log_print(log_text, color='green', attrs=['bold'])
            re_cnt = True    
    
        if re_cnt:                                
            t.tic()
            re_cnt = False
    break
    if (epoch % 2 == 0):
        save_name = os.path.join(output_dir, '{}_{}_{}.h5'.format(method,dataset_name,epoch))
        network.save_net(save_name, net)     
        #calculate error on the validation dataset 
        mae,mse = evaluate_model(save_name, data_loader_val)
        if mae < best_mae:
            best_mae = mae
            best_mse = mse
            best_model = '{}_{}_{}.h5'.format(method,dataset_name,epoch)
        log_text = 'EPOCH: %d, MAE: %.1f, MSE: %0.1f' % (epoch,mae,mse)
        log_print(log_text, color='green', attrs=['bold'])
        log_text = 'BEST MAE: %0.1f, BEST MSE: %0.1f, BEST MODEL: %s' % (best_mae,best_mse, best_model)
        log_print(log_text, color='green', attrs=['bold'])
        if use_tensorboard:
            exp.add_scalar_value('MAE', mae, step=epoch)
            exp.add_scalar_value('MSE', mse, step=epoch)
            exp.add_scalar_value('train_loss', train_loss/data_loader.get_num_samples(), step=epoch)

1545.0
tensor(32289.4766)
